# ESE 415 Final Project - Analyzing Data Scientist Salaries

## By: Abigail Alpert and Kevin Yan

# Data exploration and preprocesing

### Dataset Columns
1. **work_year** : The year the salary was paid.

2. **experience_level** : The experience level in the job during the year

3. **employment_type** : The type of employment for the role

4. **job_title** : The role worked in during the year.

5. **salary** : The total gross salary amount paid.

6. **salary_currency** : The currency of the salary paid as an ISO 4217 currency code.

7. **salaryinusd** : The salary in USD

8. **employee_residence** : Employee's primary country of residence in during the work year as an ISO 3166 country code.

9. **remote_ratio** : The overall amount of work done remotely

10. **company_location** : The country of the employer's main office or contracting branch

11. **company_size** : The median number of people that worked for the company during the year

In [84]:
import pandas as pd
import numpy as np
import re

import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer

In [85]:
df = pd.read_csv('ds_salaries.csv', index_col = 0)
df

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2020,MI,FT,Data Scientist,70000,EUR,79833,DE,0,DE,L
1,2020,SE,FT,Machine Learning Scientist,260000,USD,260000,JP,0,JP,S
2,2020,SE,FT,Big Data Engineer,85000,GBP,109024,GB,50,GB,M
3,2020,MI,FT,Product Data Analyst,20000,USD,20000,HN,0,HN,S
4,2020,SE,FT,Machine Learning Engineer,150000,USD,150000,US,50,US,L
...,...,...,...,...,...,...,...,...,...,...,...
602,2022,SE,FT,Data Engineer,154000,USD,154000,US,100,US,M
603,2022,SE,FT,Data Engineer,126000,USD,126000,US,100,US,M
604,2022,SE,FT,Data Analyst,129000,USD,129000,US,0,US,M
605,2022,SE,FT,Data Analyst,150000,USD,150000,US,100,US,M


First off, basic data cleaning (dropping duplicates and NA's).

In [86]:
df = df.drop_duplicates()
df = df.dropna(subset=['salary_in_usd']) 

Now let's create a couple flags/binary variables that could be useful for predictions

In [87]:
df['is_manager'] = df['job_title'].str.contains('Manager|Lead|Director|Head', case=False).astype(int)
df['is_remote'] = (df['remote_ratio'] == 100).astype(int)
df['is_hybrid'] = ((df['remote_ratio'] > 0) & (df['remote_ratio'] < 100)).astype(int)
df['same_country'] = (df['employee_residence'] == df['company_location']).astype(int)

In [88]:
print("Unique Job Titles:",len(standardized_df['job_title'].unique()))
print("Unique Employee Residences:",len(standardized_df['employee_residence'].unique()))
print("Unique Company Locations:",len(standardized_df['company_location'].unique()))

Unique Job Titles: 50
Unique Employee Residences: 57
Unique Company Locations: 50


## <span style="color:red"> Check if standardizing makes sense here. My concern is that it'll make our predictions hard to interpret </span>

Let's standardize the numerical features to help the model converge faster. The different numerical features have very different magnitudes which could cause issues during gradient descent.

In [89]:
num_cols = ['work_year', 'salary', 'salary_in_usd', 'remote_ratio']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), num_cols)
    ],
    remainder='passthrough'
)

X_standardized = preprocessor.fit_transform(df)

all_feature_names = list(num_cols) + [
    col for col in df.columns 
    if col not in num_cols
]

standardized_df = pd.DataFrame(X_standardized, columns=all_feature_names)

standardized_df

,work_year,salary,salary_in_usd,remote_ratio,experience_level,employment_type,job_title,salary_currency,employee_residence,company_location,company_size,is_manager,is_remote,is_hybrid,same_country
0,-1.956361,-0.167734,-0.426180,-1.710815,MI,FT,Data Scientist,EUR,DE,DE,L,0,0,0,1
1,-1.956361,-0.048869,2.068630,-1.710815,SE,FT,Machine Learning Scientist,USD,JP,JP,S,0,0,0,1
2,-1.956361,-0.158350,-0.021966,-0.487257,SE,FT,Big Data Engineer,GBP,GB,GB,M,0,0,1,1
3,-1.956361,-0.199014,-1.254701,-1.710815,MI,FT,Product Data Analyst,USD,HN,HN,S,0,0,0,1
4,-1.956361,-0.117686,0.545437,-0.487257,SE,FT,Machine Learning Engineer,USD,US,US,L,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,0.910939,-0.115183,0.600826,0.736300,SE,FT,Data Engineer,USD,US,US,M,0,1,0,1
561,0.910939,-0.132700,0.213104,0.736300,SE,FT,Data Engineer,USD,US,US,M,0,1,0,1
562,0.910939,-0.130823,0.254645,-1.710815,SE,FT,Data Analyst,USD,US,US,M,0,0,0,1
563,0.910939,-0.117686,0.545437,0.736300,SE,FT,Data Analyst,USD,US,US,M,0,1,0,1


Unfortunately, it seems like these categorical variables have many possibilities, so performing one-hot encoding on these columns may create too many columns to be feasible. We will have to find other ways to deal with these columns, as we believe that the job title will play a strong role in predicting salary.

With that being said, let's handle the lower-cardinal categorical features with one-hot encoding. Note that we ignore salary_currency as we do not think it will be predictive given we have the salary_in_usd column.

In [90]:
low_card_cat_features = [
    'experience_level',  # EN, MI, SE, EX
    'employment_type',   # FT, PT, CT, FL
    'company_size',     # S, M, L
]
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), low_card_cat_features)
    ],
    remainder='passthrough'
)

X_processed = preprocessor.fit_transform(standardized_df)


cat_encoder = preprocessor.named_transformers_['cat']
feature_names = cat_encoder.get_feature_names_out(low_card_cat_features)

all_feature_names = list(feature_names) + [
    col for col in standardized_df.columns 
    if col not in low_card_cat_features
]

processed_df = pd.DataFrame(X_processed, columns=all_feature_names)
processed_df

,experience_level_EN,experience_level_EX,experience_level_MI,experience_level_SE,employment_type_CT,employment_type_FL,employment_type_FT,employment_type_PT,company_size_L,company_size_M,...,salary_in_usd,remote_ratio,job_title,salary_currency,employee_residence,company_location,is_manager,is_remote,is_hybrid,same_country
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,-0.426180,-1.710815,Data Scientist,EUR,DE,DE,0,0,0,1
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,2.068630,-1.710815,Machine Learning Scientist,USD,JP,JP,0,0,0,1
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,-0.021966,-0.487257,Big Data Engineer,GBP,GB,GB,0,0,1,1
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,-1.254701,-1.710815,Product Data Analyst,USD,HN,HN,0,0,0,1
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.545437,-0.487257,Machine Learning Engineer,USD,US,US,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.600826,0.736300,Data Engineer,USD,US,US,0,1,0,1
561,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.213104,0.736300,Data Engineer,USD,US,US,0,1,0,1
562,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.254645,-1.710815,Data Analyst,USD,US,US,0,0,0,1
563,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.545437,0.736300,Data Analyst,USD,US,US,0,1,0,1


## <span style="color:red"> we need to figure out how we'll handle the job_title and employee_residence columns </span>

**job_title:** this feaure seems like it will be important, as we suspect samples with the same/similar job title to have more similar salaries.

**employee_residence:** this feature seems important, because salary often relates to cost of living (ie: the same position at the same company will pay different amounts depending on which US city it is located in).

## <span style="color:limegreen"> *for now, let's see what happens in we use label encoding* </span>

In [91]:
# label_encoder = LabelEncoder()

# label_df = processed_df.copy()
# label_df['job_title']= label_encoder.fit_transform(label_df['job_title'])
# # label_df['employee_residence']=label_encoder.fit_transform(label_df['employee_residence'])

## <span style="color:limegreen"> *okay lets try "Bag of Words" encoding job descriptions* </span>

In [92]:
processed_df["job_title_clean"] = processed_df["job_title"].str.lower().apply(lambda x: re.sub(r'[^\w\s]', '', x))

unique_words = (processed_df['job_title_clean']
                .str.split()  # Split into words (by whitespace)
                .explode()    # Create one row per word
                .unique()    # Get unique values
                .tolist())
print(unique_words)
print("---")
print(len(unique_words))

['data', 'scientist', 'machine', 'learning', 'big', 'engineer', 'product', 'analyst', 'lead', 'business', 'science', 'consultant', 'bi', 'director', 'of', 'research', 'manager', 'engineering', 'infrastructure', 'ml', 'ai', 'computer', 'vision', 'principal', 'head', '3d', 'researcher', 'analytics', 'applied', 'marketing', 'cloud', 'financial', 'software', 'developer', 'specialist', 'architect', 'finance', 'staff', 'etl', 'nlp']
---
40


It seems like there are still a lot of possible words in the job titles. Let's normalize a couple ("ml" -> "machine" and "learning", "computer" and "vision" -> "computer_vision"

Then we will apply BoW encoding for the top 20 title-words.

In [93]:
import pandas as pd
from collections import Counter
from nltk.corpus import stopwords
import re
import nltk

nltk.download('stopwords')

def normalize_terms(text):
    """
    Replace specific terms/phrases before tokenization.
    """
    # Convert "ml" to "machine learning" (case-insensitive)
    text = re.sub(r'\bml\b', 'machine learning', text, flags=re.IGNORECASE)
    
    # Combine "computer vision" as one term (handle with/without hyphen/space)
    text = re.sub(r'\bcomputer[-\s]?vision\b', 'computer_vision', text, flags=re.IGNORECASE)
    return text

def get_word_frequency_df(df, column_name, min_count=0, custom_stopwords=None):
    """
    Create a DataFrame of word frequencies with term normalization.
    """
    # Combine all text, normalize terms, and lowercase
    all_text = ' '.join(df[column_name].dropna().astype(str))
    all_text = normalize_terms(all_text).lower()
    
    # Tokenize words (hyphen/underscore-aware)
    words = re.findall(r'\b[a-z_]+(?:\-[a-z_]+)*\b', all_text)  # Matches hyphenated/underscore terms
    
    # Get stopwords and add custom ones
    stop_words = set(stopwords.words('english'))
    if custom_stopwords:
        stop_words.update([w.lower() for w in custom_stopwords])
    
    # Count word frequencies
    word_counts = Counter(words)
    
    # Filter words
    filtered_counts = {
        word: count for word, count in word_counts.items() 
        if word not in stop_words and count > min_count
    }
    
    # Create DataFrame
    word_df = (pd.DataFrame.from_dict(filtered_counts, orient='index', columns=['count'])
               .reset_index()
               .rename(columns={'index': 'word'})
               .sort_values('count', ascending=False))
    
    return word_df

word_freq_df = get_word_frequency_df(processed_df, 'job_title')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kevinyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [94]:
top_20_words = word_freq_df.head(20)['word'].tolist()


def bow_encode(title, word_list):
    title = title.lower()
    features = {}
    for word in word_list:
        # Handle underscores (e.g., "machine_learning" → "machine learning")
        search_term = word.replace('_', ' ')
        features[f"bow_{word}"] = 1 if re.search(rf'\b{search_term}\b', title) else 0
    return features

# Apply to each job title and create a DataFrame
bow_features = processed_df['job_title'].apply(lambda x: bow_encode(x, top_20_words)).apply(pd.Series)

new_df = pd.concat([processed_df, bow_features], axis=1)

In [111]:
new_df

,experience_level_EN,experience_level_EX,experience_level_MI,experience_level_SE,employment_type_CT,employment_type_FL,employment_type_FT,employment_type_PT,company_size_L,company_size_M,...,bow_principal,bow_architect,bow_computer_vision,bow_head,bow_director,bow_big,bow_applied,bow_ai,bow_engineering,employee_residence_mean_sal
0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,-0.349970
1,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,-0.097936
2,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0,0,0,0,0,1,0,0,0,-0.403512
3,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,-1.254701
4,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0,0,0,0,0,0,0,0,0,0.546751
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0.546751
561,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0.546751
562,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0.546751
563,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0,0,0,0,0,0,0,0,0,0.546751


## <span style="color:limegreen"> *lets try target encoding for the employee_residence column* </span>

In [96]:
new_df['salary_in_usd']

0     -0.426180
1      2.068630
2     -0.021966
3     -1.254701
4      0.545437
         ...   
560    0.600826
561    0.213104
562    0.254645
563    0.545437
564    1.237797
Name: salary_in_usd, Length: 565, dtype: object

In [102]:
# Calculate mean salary per country for 'employee_residence'
new_df['salary_in_usd'] = pd.to_numeric(new_df['salary_in_usd'])
mean_employee_residence = new_df.groupby('employee_residence')['salary_in_usd'].mean().to_dict()
new_df['employee_residence_mean_sal'] = new_df['employee_residence'].map(mean_employee_residence)

In [117]:

new_df = new_df.drop(columns=['job_title_clean'])

# 2. Convert all object columns to integers (with error handling)
for col in new_df.select_dtypes(include=['object']).columns:
    try:
        new_df[col] = pd.to_numeric(new_df[col], errors='raise').astype(int)
    except ValueError as e:
        print(f"Could not convert column '{col}' to int: {e}")
        # Alternative: Convert to categorical codes if conversion fails
        new_df[col] = new_df[col].astype('category').cat.codes


KeyError: "['job_title_clean'] not found in axis"

The last this to do before model building is to split the features from the target variable. As mentioned earlier, our target will be "salary_in_usd".

In [119]:
target = 'salary_in_usd'
features = [col for col in new_df.columns if col not in target]

# X = label_df[features]
# y = label_df['salary_in_usd']

X = new_df[features]
y = new_df['salary_in_usd']

# Model Building

We are going to fit our model using gradient descent, so let's start by implmenting a multivariate linear GD algorithm with MSE as the loss function

### Vanilla GD

In [120]:
def gradient_descent(grd_func, gamma, w0, N, tol=1e-05):
    '''This function performs N-many iterations of gradient descent
    
    Args:
        grd_func: the gradient of the function to minimize
        gamma: the stepsize
        w0: the starting x value/vector
        N: the maximum number of iterations
        (*) tol: the convergence tolerance

    Output:
        w: the final weight vector
    '''

    w = w0 # initialize w
    
    for n in range(N): # perform gradient descent N many times
        grad = grd_func(w)

        if np.linalg.norm(grad) < tol: # check if the function has converged (and return early if so)
            print(f"Converged!\nIteration: {n}")
            return w

        x_new = w - gamma*grad # gradient update rule
        w = x_new #update w

    print(f"Did not converge...\n Final Result: {w}")
    return w

For now, we are using vanilla GD (ie a constant stepsize), although we might want to consider updating this if convergence takes too long... Let's see how it works

In [121]:
# Until we figure out a way to encode job_title, salary_currency, employee_residence, and company_location, we will drop those columns...
cols_to_drop = ['job_title', 'salary_currency', 'employee_residence', 'company_location']

X_subset = X.drop(columns=cols_to_drop)


In [122]:
X_subset.dtypes

experience_level_EN              int64
experience_level_EX              int64
experience_level_MI              int64
experience_level_SE              int64
employment_type_CT               int64
employment_type_FL               int64
employment_type_FT               int64
employment_type_PT               int64
company_size_L                   int64
company_size_M                   int64
company_size_S                   int64
work_year                        int64
remote_ratio                     int64
is_manager                       int64
is_remote                        int64
is_hybrid                        int64
same_country                     int64
bow_data                         int64
bow_engineer                     int64
bow_scientist                    int64
bow_analyst                      int64
bow_machine                      int64
bow_learning                     int64
bow_science                      int64
bow_manager                      int64
bow_research             

In [123]:
def f_linear(X, w, b):
    return w.T @ X + b
X_aug = np.hstack([X_subset, np.ones((X_subset.shape[0], 1))]) # adding a column for the bias term

def grad_mse_linear(w_aug):
    n = X_aug.shape[0]
    return (2/n) * X_aug.T @ (X_aug @ w_aug - y)

w0_aug = np.ones(X_aug.shape[1])

# Run gradient descent
w_star_aug = gradient_descent(grad_mse_linear, gamma=0.01, w0=w0_aug, N=100000)

# Extract weights and bias
w_star = w_star_aug[:-1]
b_star = w_star_aug[-1]
print(f'w_star: {w_star}')
print(f'b_star: {b_star}')

Did not converge...
 Final Result: [-0.01486895  1.3929511   0.25392868  0.51683905  0.9385712   0.17032283
  0.56136875  0.4785871   0.50535044  0.37335352  0.27014592 -0.1693025
  0.2559406  -0.51868861 -0.22926896 -0.36594056 -0.06682747 -0.08177644
 -0.15226334 -0.11588197 -0.50859516  0.05626862  0.05626862 -0.07025089
  0.47464574  0.28668774 -0.09946633  0.87588797  1.1781831   0.34635936
 -0.12845881  0.27600506  0.85477263 -0.08017658  0.75361968  0.22238736
 -0.26031961  0.88984269 -0.85115012]
w_star: [-0.01486895  1.3929511   0.25392868  0.51683905  0.9385712   0.17032283
  0.56136875  0.4785871   0.50535044  0.37335352  0.27014592 -0.1693025
  0.2559406  -0.51868861 -0.22926896 -0.36594056 -0.06682747 -0.08177644
 -0.15226334 -0.11588197 -0.50859516  0.05626862  0.05626862 -0.07025089
  0.47464574  0.28668774 -0.09946633  0.87588797  1.1781831   0.34635936
 -0.12845881  0.27600506  0.85477263 -0.08017658  0.75361968  0.22238736
 -0.26031961  0.88984269]
b_star: -0.85115011

Using vanilla GD is taking too long to converge. Thus, we should update the algorithm

<span style="color:red"> perhaps this is another thing to talk to Ben about. Is there standard practice for picking which gradient method to use?</span>

### SGD (mini-batch)

In [124]:
def stochastic_gd(grd_func, gamma, w0, X, y, N, batch_size, tol=1e-5):
    '''This function performs N-many iterations of stochastic gradient descent (SGD) for a model.
    
    Args:
        grd_func: the gradient of the function to minimize
        gamma: the stepsize
        w0: the starting weight vector
        X: the input data (Pandas DataFrame)
        y: the output data (numpy array)
        N: maximum number of iterations
        batch_size: size of mini-batch for each SGD update
        (*) tol: convergence tolerance

    Output:
        w: final weight vector after training
    '''
    n = X.shape[0]  # number of samples
    w = w0  # initialize weight vector

    for i in range(N):
        # Shuffle data
        indices = np.random.permutation(n)
        X_shuffled = X[indices,:]
        y_shuffled = y[indices]

        for j in range(0, n, batch_size):
            X_batch = X_shuffled[j:j + batch_size,:]
            y_batch = y_shuffled[j:j + batch_size]

            # Compute gradients using the user-defined gradient function
            grad = grd_func(w, X_batch, y_batch)

            # Update weights using the gradient and learning rate
            w_new = w - gamma * grad
            w = w_new

        # Convergence check (optional early stopping)
        if np.linalg.norm(grad) < tol:
            print(f"Converged!\nEpoch: {i}")
            return w

    print(f"Did not converge...\nFinal Result: {w}")
    return w

In [125]:
def sgd_grad_linear_mse(w, X_batch, y_batch):
    X_aug = np.hstack([X_batch, np.ones((X_batch.shape[0], 1))])  # Adding bias as the last column
    n = len(X_aug)
        
    y_pred = X_aug @ w
    error = y_batch - y_pred
    
    return -(2/n) * X_aug.T @ error  # gradient with respect to w (including bias)

w0_aug = np.ones(X_aug.shape[1]+1)

w_star_aug = stochastic_gd(sgd_grad_linear_mse, gamma=0.01, w0=w0_aug, X=X_aug, y=y, N=100000, batch_size=25)

# Extract weights and bias
w_star = w_star_aug[:-1]
b_star = w_star_aug[-1]
print(f'w_star: {w_star}')
print(f'b_star: {b_star}')


Did not converge...
Final Result: [ 0.05348536  1.46166506  0.32107977  0.5828616   1.00568508  0.23887639
  0.62907649  0.54545384  0.59443099  0.46313825  0.36152255 -0.16683236
  0.16623192 -0.51728671 -0.14007761 -0.27459868 -0.06764125 -0.08420403
 -0.15503047 -0.11868368 -0.51082671  0.05594761  0.05594761 -0.07204006
  0.4766052   0.28324074 -0.10359077  0.87724616  1.18043806  0.34193661
 -0.1350558   0.27409086  0.85477107 -0.08136638  0.7507038   0.22196415
 -0.26355959  0.88791162 -0.58090821 -0.58090821]
w_star: [ 0.05348536  1.46166506  0.32107977  0.5828616   1.00568508  0.23887639
  0.62907649  0.54545384  0.59443099  0.46313825  0.36152255 -0.16683236
  0.16623192 -0.51728671 -0.14007761 -0.27459868 -0.06764125 -0.08420403
 -0.15503047 -0.11868368 -0.51082671  0.05594761  0.05594761 -0.07204006
  0.4766052   0.28324074 -0.10359077  0.87724616  1.18043806  0.34193661
 -0.1350558   0.27409086  0.85477107 -0.08136638  0.7507038   0.22196415
 -0.26355959  0.88791162 -0.5809

### AGM

In [126]:
def accelerated_gd(grd_func, gamma, w0, N, momentum=0.9, tol=1e-05):
    '''This function performs N-many iterations of momentum-based gradient descent
    
    Args:
        grd_func: the gradient of the function to minimize
        gamma: the stepsize
        w0: the starting x value/vector
        N: the maximum number of iterations
        (*) momentum: the momentum parameter (usually between 0 and 1)
        (*) tol: the convergence tolerance

    Output:
        w: the final weight vector
    '''

    w = w0
    v = np.zeros_like(w0)  # initialize velocity (same shape as w0)
    
    for n in range(N):
        grad = grd_func(w)

        if np.linalg.norm(grad) < tol:
            print(f"Converged!\nIteration: {n}")
            return w

        v = momentum * v - gamma * grad
        w = w + v 

    print(f"Did not converge...\n Final Result: {w}")
    return w

In [127]:
def f_linear(X, w, b):
    return w.T @ X + b

X_aug = np.hstack([X_subset, np.ones((X_subset.shape[0], 1))]) # adding a column for the bias term

def grad_mse_linear(w_aug):
    n = X_aug.shape[0]
    return (2/n) * X_aug.T @ (X_aug @ w_aug - y)

w0_aug = np.ones(X_aug.shape[1])

# Run gradient descent
w_star_aug = accelerated_gd(grad_mse_linear, gamma=0.01, w0=w0_aug, N=100000)

# Extract weights and bias
w_star = w_star_aug[:-1]
b_star = w_star_aug[-1]
print(f'w_star: {w_star}')
print(f'b_star: {b_star}')

Converged!
Iteration: 13176
w_star: [-0.01431122  1.39356285  0.25440479  0.51739103  0.93872312  0.17136587
  0.56192314  0.47903531  0.50611434  0.37411987  0.27081324 -0.16929021
  0.25517456 -0.51940712 -0.22851653 -0.36526147 -0.06705932 -0.08385855
 -0.15470895 -0.11831094 -0.51109861  0.05512345  0.05512345 -0.07192446
  0.47456307  0.28435558 -0.10109257  0.87662969  1.17818014  0.34351852
 -0.13132437  0.27468874  0.85471139 -0.0802301   0.75363027  0.21957225
 -0.26248149  0.88981469]
b_star: -0.8489525557829253
